In [1]:
import requests
import schedule
import time
import json
import csv
import os
from pathlib import Path

# 오피넷 API Key
api_key = "F241014357"

# 운영체제에 따른 바탕화면 경로 설정
def get_desktop_path():
    if os.name == 'nt':  # Windows
        return os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')
    else:  # Mac OS 또는 Linux
        return str(Path.home() / 'Desktop')

desktop_path = get_desktop_path()
# 유종코드를 한국어로 변환하는 함수
def convert_fuel_type(fuel_code):
    fuel_type_map = {
        "B027": "휘발유",
        "D047": "경유",
        "B034": "고급휘발유",
        "K015": "자동차부탄"
    }
    return fuel_type_map.get(fuel_code, "기타")  # 예기치 않은 값은 "기타"로 처리

# API 요청 함수 정의
def request_api(url, params):
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # HTTP 에러 발생 시 예외 발생
        return response.json()  # JSON 형식 응답 반환
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP 에러 발생: {http_err}")
    except Exception as err:
        print(f"기타 에러 발생: {err}")
    return None

# CSV 파일 저장 함수 (파일이 있으면 추가, 없으면 새로 생성)
def save_to_csv(all_data, filename):
    fieldnames = ["제품_구분", "고유ID", "유종코드", "주유소명", "가격", "거리", "GIS_X_좌표", "GIS_Y_좌표"]
    try:
        file_path = os.path.join(desktop_path, filename)
        
        # 파일이 존재하는지 확인하여, 존재하면 append 모드로 열고, 없으면 새로 생성
        file_exists = os.path.exists(file_path)

        with open(file_path, mode="a", newline="", encoding="utf-8-sig") as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            
            # 첫 번째로 파일을 만들 때는 헤더를 추가
            if not file_exists:
                writer.writeheader()

            writer.writerows(all_data)
        print(f"데이터가 {file_path} 파일로 저장되었습니다.")
    except Exception as e:
        print(f"CSV 파일 저장 중 오류 발생: {e}")

# 9. 반경 내 주유소 검색
def search_stations_within_radius():
    url = "http://www.opinet.co.kr/api/aroundAll.do"
    all_data = []  # 모든 데이터를 모을 리스트

    # 제품 구분과 정렬 기준 목록
    prodcd_options = {
        "B027": "휘발유",
        "D047": "경유",
        "B034": "고급휘발유",
        "K015": "자동차부탄"
    }
    sort_options = {
        "1": "가격순",
        "2": "거리순"
    }

    # 모든 조합에 대해 요청 수행
    for prodcd, prodcd_name in prodcd_options.items():
        for sort, sort_name in sort_options.items():
            params = {
                "code": api_key,       # 필수: API 키
                "out": "json",        # 필수: 정보 노출 형식 (json)
                "x": "314996.15900",  # 필수: 기준 위치 X좌표 (예: 서울)
                "y": "544938.50758",  # 필수: 기준 위치 Y좌표 (예: 서울)
                "radius": "1000",     # 필수: 반경 (1000m)
                "prodcd": prodcd,     # 필수: 제품 구분
                "sort": sort          # 필수: 정렬 기준
            }

            data = request_api(url, params)
            if data:
                result_data = data.get("RESULT", {}).get("OIL", []) or []

                print(f"\n[제품: {prodcd_name}, 정렬 기준: {sort_name}]")
                print(json.dumps(result_data, indent=4, ensure_ascii=False))

                # 데이터를 all_data 리스트에 추가
                for station in result_data:
                    all_data.append({
                        "제품_구분": prodcd_name,  # 제품 구분 추가
                        "고유ID": station.get("UNI_ID", ""),
                        "유종코드": convert_fuel_type(station.get("POLL_DIV_CD", "")),  # 유종코드를 한국어로 변환
                        "주유소명": station.get("OS_NM", ""),
                        "가격": station.get("PRICE", ""),
                        "거리": station.get("DISTANCE", ""),
                        "GIS_X_좌표": station.get("GIS_X_COOR", ""),
                        "GIS_Y_좌표": station.get("GIS_Y_COOR", "")
                    })
            else:
                print(f"\n[제품: {prodcd_name}, 정렬 기준: {sort_name}] - 데이터를 가져오는 데 실패했습니다.")

    # 모든 데이터를 하나의 CSV 파일에 저장
    save_to_csv(all_data, "station_all.csv")

# 주기적으로 주유소 상세정보 가져오기 실행
schedule.every(1).minutes.do(search_stations_within_radius)

# 스케줄러 실행
try:
    while True:
        schedule.run_pending()
        time.sleep(1)
except KeyboardInterrupt:
    print("프로그램이 종료되었습니다.")

프로그램이 종료되었습니다.


In [2]:
!pip install schedule